In [1]:
import pandas as pd
import ast

useful_cols = ['parameters', 'algorithm runtime', 'algorithm cost time', 'algorithm index creation time', 'algorithm created #indexes','index create time', 'cost requests','cost estimate', 'indexed columns']

filtered_df = pd.read_csv("/home/peijingx/repos/index_selection_evaluation/benchmark_results/results_extend_filtered_custom_tpch_78_queries.csv", sep=';', usecols = useful_cols)
filtered_df['parameters'] = filtered_df['parameters'].apply(ast.literal_eval)

extract_clause = lambda x: x['candidate_file'].split('_')[-1][:-5]
filtered_df['clauses'] = filtered_df['parameters'].apply(extract_clause)
filtered_df.drop('parameters', axis = 1, inplace=True)

In [2]:
experiment = ast.literal_eval( 
  filtered_df.sort_values('cost estimate')[['indexed columns', 'clauses']].iloc[0]['indexed columns']
)

for i in experiment:
  print(f'create index {i};')

create index customer(c_custkey,c_nationkey,c_address);
create index lineitem(l_orderkey);
create index lineitem(l_orderkey,l_receiptdate);
create index lineitem(l_partkey,l_shipdate,l_suppkey);
create index lineitem(l_receiptdate);
create index lineitem(l_shipdate);
create index orders(o_custkey,o_orderdate,o_orderkey,o_shippriority);
create index orders(o_orderdate);
create index orders(o_orderkey,o_orderpriority);
create index part(p_partkey);
create index partsupp(ps_partkey);


In [3]:
experiment

['customer(c_custkey,c_nationkey,c_address)',
 'lineitem(l_orderkey)',
 'lineitem(l_orderkey,l_receiptdate)',
 'lineitem(l_partkey,l_shipdate,l_suppkey)',
 'lineitem(l_receiptdate)',
 'lineitem(l_shipdate)',
 'orders(o_custkey,o_orderdate,o_orderkey,o_shippriority)',
 'orders(o_orderdate)',
 'orders(o_orderkey,o_orderpriority)',
 'part(p_partkey)',
 'partsupp(ps_partkey)']

In [10]:
filtered_df[filtered_df["clauses"]=='w'].sort_values('cost estimate')['indexed columns'].iloc[0]

'["lineitem(l_orderkey)", "lineitem(l_partkey,l_shipdate,l_suppkey)", "lineitem(l_receiptdate)", "lineitem(l_shipdate)", "orders(o_custkey,o_orderdate,o_orderkey)", "orders(o_orderdate)", "orders(o_orderkey)", "part(p_partkey)", "partsupp(ps_partkey)"]'

In [12]:
best_unfiltered = ["customer(c_custkey,c_nationkey,c_mktsegment,c_acctbal)", "lineitem(l_orderkey)", "lineitem(l_orderkey,l_receiptdate)", "lineitem(l_partkey,l_shipdate,l_suppkey)", "lineitem(l_receiptdate)", "lineitem(l_shipdate)", "orders(o_custkey,o_orderdate,o_orderkey,o_shippriority)", "orders(o_orderdate)", "orders(o_orderkey,o_orderpriority)", "part(p_partkey,p_type,p_name)", "partsupp(ps_partkey)"]
best_w = ast.literal_eval(filtered_df[filtered_df["clauses"]=='w'].sort_values('cost estimate')['indexed columns'].iloc[0])
best_ws = ast.literal_eval(filtered_df[filtered_df["clauses"]=='ws'].sort_values('cost estimate')['indexed columns'].iloc[0])
best_wg = ast.literal_eval(filtered_df[filtered_df["clauses"]=='wg'].sort_values('cost estimate')['indexed columns'].iloc[0])
best_wgs = ast.literal_eval(filtered_df[filtered_df["clauses"]=='wgs'].sort_values('cost estimate')['indexed columns'].iloc[0])



In [18]:
cols_unfiltered = set([a for l in best_unfiltered for a in l.split('(')[1][:-1].split(",")])
cols_w = set([a for l in best_w for a in l.split('(')[1][:-1].split(",")])
cols_ws = set([a for l in best_ws for a in l.split('(')[1][:-1].split(",")])
cols_wg = set([a for l in best_wg for a in l.split('(')[1][:-1].split(",")])
cols_wgs = set([a for l in best_wgs for a in l.split('(')[1][:-1].split(",")])

In [32]:
set([l for l in best_unfiltered])
set([l for l in best_unfiltered])

{'customer(c_custkey,c_nationkey,c_mktsegment,c_acctbal)',
 'lineitem(l_orderkey)',
 'lineitem(l_orderkey,l_receiptdate)',
 'lineitem(l_partkey,l_shipdate,l_suppkey)',
 'lineitem(l_receiptdate)',
 'lineitem(l_shipdate)',
 'orders(o_custkey,o_orderdate,o_orderkey,o_shippriority)',
 'orders(o_orderdate)',
 'orders(o_orderkey,o_orderpriority)',
 'part(p_partkey,p_type,p_name)',
 'partsupp(ps_partkey)'}

In [27]:
print(cols_unfiltered.difference(cols_w))
print(cols_w.difference(cols_unfiltered))

{'c_acctbal', 'o_shippriority', 'c_mktsegment', 'c_custkey', 'c_nationkey', 'p_type', 'p_name', 'o_orderpriority'}
set()


In [28]:
print(cols_unfiltered.difference(cols_wg))
print(cols_wg.difference(cols_unfiltered))

{'c_acctbal', 'c_mktsegment', 'p_type', 'p_name'}
{'c_address'}


In [29]:
print(cols_unfiltered.difference(cols_ws))
print(cols_ws.difference(cols_unfiltered))

{'c_acctbal', 'o_shippriority', 'c_nationkey', 'p_type', 'p_name'}
set()


In [30]:
print(cols_unfiltered.difference(cols_wgs))
print(cols_wgs.difference(cols_unfiltered))

{'c_acctbal', 'c_mktsegment', 'p_type', 'p_name'}
{'c_address'}


In [31]:
cols_all = cols_wgs.union(cols_w, cols_wg, cols_ws)
print(cols_unfiltered.difference(cols_all))
print(cols_all.difference(cols_unfiltered))

{'c_acctbal', 'p_type', 'p_name'}
{'c_address'}
